# Importing necessary libraries

In [0]:
#requets library helps us to access required content through urls
import requests

#pandas library helps us in converting data in variious formats to dataframes which are easier to work with
import pandas as pd

#Folium library helps in working with maps
import folium

#for converting an address into latitude and longitude values
from geopy.geocoders import Nominatim

import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# Retrieve data stored in google drive into a pandas datafame.
#### One can find the original file in the link provided in the report

In [0]:
path='/content/drive/My Drive/crime.csv'
crime=pd.read_csv(path, sep=',', engine='python')
crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182080058,2403,Disorderly Conduct,DISTURBING THE PEACE,E18,495,NaN,2018-10-03 20:13:00,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,"(42.26260773, -71.12118637)"
1,I182080053,3201,Property Lost,PROPERTY - LOST,D14,795,NaN,2018-08-30 20:00:00,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,"(42.35211146, -71.13531147)"
2,I182080052,2647,Other,THREATS TO DO BODILY HARM,B2,329,NaN,2018-10-03 19:20:00,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,"(42.30812619, -71.07692974)"
3,I182080051,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,92,NaN,2018-10-03 20:00:00,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,"(42.35945371, -71.05964817)"
4,I182080050,3122,Aircraft,AIRCRAFT INCIDENTS,A7,36,NaN,2018-10-03 20:49:00,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,"(42.37525782, -71.02466343)"


# Analyzing Data

In [0]:
# To obtain total no.of streets with repective no.of crime entries
crime['STREET'].nunique()


4684

 There are totally **4684** rows. let's deal with the high crime rate areas so the end user can avoid them.

In [0]:
high_crime=crime['STREET'].value_counts().to_frame()

#To extract top 30 crime rated areas
high_crime=high_crime.head(30)
high_crime.reset_index(inplace=True)
high_crime.columns=['STREET','CRIME_COUNT']

#collect the co-ordinates of these 30 locations
high_crime=high_crime.merge(crime,on=['STREET'])
high_crime=high_crime[['STREET','CRIME_COUNT','Lat','Long']]

#When we merged data all the thousands of rows will be merged with it.This increases API calls
#So we shall remove these duplicates
high_crime=high_crime.drop_duplicates('STREET')
high_crime.sort_values(by='STREET',inplace=True,ascending=True)
high_crime.reset_index(inplace=True)
high_crime.head()

,index,STREET,CRIME_COUNT,Lat,Long
0,78683,ADAMS ST,2104,42.305378,-71.061514
1,93191,ALBANY ST,1466,42.330769,-71.077665
2,94657,ALLSTATE RD,1332,42.326966,-71.061986
3,89858,AMERICAN LEGION HWY,1672,42.297950,-71.087744
4,80787,BEACON ST,2016,42.355052,-71.073907


In [0]:
#lets keep only crime data that is relevant for analysis
streets=high_crime['STREET']
crime=crime[crime['STREET'].isin(streets)]
crime.reset_index(drop=True,inplace=True)
crime=crime[['STREET','OFFENSE_CODE_GROUP','Lat','Long']]
crime.head()

,STREET,OFFENSE_CODE_GROUP,Lat,Long
0,CAMBRIDGE ST,Aggravated Assault,42.359454,-71.059648
1,DORCHESTER AVE,Vandalism,42.299197,-71.060470
2,TREMONT ST,Verbal Disputes,42.333807,-71.103778
3,COMMONWEALTH AVE,Towed,42.348866,-71.089363
4,COLUMBIA RD,Motor Vehicle Accident Response,42.330015,-71.038513


k-means cluster algorithm works only with numeric data. But the crime category is a string. Using get_dummies method in pandas we can create a separate column for each category that depicts its row value with 0 or 1. This is called one-hot encoding

In [0]:
crime_k=pd.get_dummies(crime[['OFFENSE_CODE_GROUP']],prefix="",prefix_sep="")
crime_k['STREET']=crime['STREET']

#move street column to starting
cols=[crime_k.columns[-1]]+list(crime_k.columns[:-1])
crime_k=crime_k[cols]
crime_k.head()

,STREET,Aggravated Assault,Aircraft,Arson,Assembly or Gathering Violations,Auto Theft,Auto Theft Recovery,Ballistics,Biological Threat,Bomb Hoax,Burglary - No Property Taken,Commercial Burglary,Confidence Games,Counterfeiting,Criminal Harassment,Disorderly Conduct,Drug Violation,Embezzlement,Evading Fare,Explosives,Fire Related Reports,Firearm Discovery,Firearm Violations,Fraud,Gambling,HOME INVASION,HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE,Harassment,Harbor Related Incidents,Homicide,INVESTIGATE PERSON,Investigate Person,Investigate Property,Landlord/Tenant Disputes,Larceny,Larceny From Motor Vehicle,License Plate Related Incidents,License Violation,Liquor Violation,Manslaughter,Medical Assistance,Missing Person Located,Missing Person Reported,Motor Vehicle Accident Response,Offenses Against Child / Family,Operating Under the Influence,Other,Other Burglary,Phone Call Complaints,Police Service Incidents,Prisoner Related Incidents,Property Found,Property Lost,Property Related Damage,Prostitution,Recovered Stolen Property,Residential Burglary,Restraining Order Violations,Robbery,Search Warrants,Service,Simple Assault,Towed,Vandalism,Verbal Disputes,Violations,Warrant Arrests
0,CAMBRIDGE ST,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DORCHESTER AVE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,TREMONT ST,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,COMMONWEALTH AVE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,COLUMBIA RD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


let's find mean of each category group of crimes presence to obtain a normalized value and it decreases no.of data rows

In [0]:
crime_grouped=crime_k.groupby(by=['STREET']).mean().reset_index()
print(crime_grouped.shape)
crime_grouped.head()

(30, 67)


,STREET,Aggravated Assault,Aircraft,Arson,Assembly or Gathering Violations,Auto Theft,Auto Theft Recovery,Ballistics,Biological Threat,Bomb Hoax,Burglary - No Property Taken,Commercial Burglary,Confidence Games,Counterfeiting,Criminal Harassment,Disorderly Conduct,Drug Violation,Embezzlement,Evading Fare,Explosives,Fire Related Reports,Firearm Discovery,Firearm Violations,Fraud,Gambling,HOME INVASION,HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE,Harassment,Harbor Related Incidents,Homicide,INVESTIGATE PERSON,Investigate Person,Investigate Property,Landlord/Tenant Disputes,Larceny,Larceny From Motor Vehicle,License Plate Related Incidents,License Violation,Liquor Violation,Manslaughter,Medical Assistance,Missing Person Located,Missing Person Reported,Motor Vehicle Accident Response,Offenses Against Child / Family,Operating Under the Influence,Other,Other Burglary,Phone Call Complaints,Police Service Incidents,Prisoner Related Incidents,Property Found,Property Lost,Property Related Damage,Prostitution,Recovered Stolen Property,Residential Burglary,Restraining Order Violations,Robbery,Search Warrants,Service,Simple Assault,Towed,Vandalism,Verbal Disputes,Violations,Warrant Arrests
0,ADAMS ST,0.017110,0.0,0.0,0.000951,0.016635,0.004753,0.005228,0.0,0.000000,0.000000,0.005228,0.013783,0.009506,0.000475,0.006179,0.048004,0.000475,0.000951,0.000000,0.006654,0.001901,0.003802,0.011882,0.000475,0.0,0.0,0.007605,0.0,0.000475,0.0,0.048479,0.031844,0.000951,0.040875,0.027567,0.003327,0.007129,0.001426,0.000000,0.074620,0.011882,0.005228,0.213403,0.001426,0.004278,0.057034,0.000951,0.0,0.007129,0.000475,0.011407,0.027091,0.003327,0.000475,0.004278,0.014259,0.005703,0.015209,0.002376,0.001901,0.040399,0.015209,0.046103,0.050856,0.032319,0.028992
1,ALBANY ST,0.029332,0.0,0.0,0.000682,0.006139,0.000682,0.001364,0.0,0.000682,0.000000,0.007503,0.004775,0.000682,0.000000,0.016371,0.156889,0.001364,0.001364,0.000000,0.001364,0.000000,0.006821,0.008186,0.000000,0.0,0.0,0.004775,0.0,0.000000,0.0,0.035471,0.011596,0.000682,0.070941,0.013643,0.000000,0.000682,0.006139,0.000000,0.053206,0.007503,0.005457,0.160300,0.002046,0.004093,0.090723,0.003411,0.0,0.013643,0.001364,0.008186,0.025921,0.003411,0.001364,0.009550,0.000682,0.002729,0.015007,0.000000,0.000000,0.077763,0.006821,0.015007,0.002729,0.015689,0.085266
2,ALLSTATE RD,0.014264,0.0,0.0,0.000000,0.009760,0.003754,0.000000,0.0,0.000000,0.000000,0.002252,0.026276,0.002252,0.000000,0.003754,0.039039,0.006006,0.001502,0.000000,0.000751,0.000000,0.002252,0.008258,0.000000,0.0,0.0,0.003754,0.0,0.000000,0.0,0.044294,0.006006,0.000000,0.337838,0.030030,0.001502,0.001502,0.000000,0.000000,0.045796,0.000751,0.000751,0.122372,0.000751,0.000751,0.050300,0.000751,0.0,0.011261,0.001502,0.006757,0.059309,0.002252,0.000000,0.009760,0.000000,0.000751,0.006006,0.000000,0.000751,0.028529,0.002252,0.015766,0.003754,0.012012,0.072072
3,AMERICAN LEGION HWY,0.016746,0.0,0.0,0.000598,0.009569,0.000000,0.001196,0.0,0.000000,0.000598,0.006579,0.003589,0.016746,0.000598,0.004187,0.041866,0.007775,0.001794,0.000000,0.005383,0.000598,0.007177,0.014354,0.000000,0.0,0.0,0.007177,0.0,0.000000,0.0,0.046053,0.025120,0.002990,0.122010,0.026316,0.001196,0.001794,0.000000,0.000598,0.044856,0.004785,0.002392,0.254187,0.002392,0.000598,0.063397,0.002392,0.0,0.002392,0.001196,0.009569,0.046651,0.001794,0.000000,0.001794,0.000598,0.001794,0.005981,0.000598,0.000598,0.043660,0.013158,0.036483,0.024522,0.048445,0.013158
4,BEACON ST,0.011905,0.0,0.0,0.026786,0.017361,0.004960,0.000496,0.0,0.000000,0.000000,0.003472,0.009921,0.003968,0.000496,0.008433,0.026786,0.001488,0.001488,0.000496,0.008433,0.000992,0.001984,0.026786,0.000000,0.0,0.0,0.009425,0.0,0.000000,0.0,0.038194,0.042163,0.001984,0.083829,0.089782,0.001488,0.008929,0.003968,0.000000,0.057540,0.002480,0.000496,0.171131,0.000000,0.001984,0.033730,0.001984,0.0,0.004960,0.000000,0.021329,0.033730,0.003968,0.000000,0.002976,0.015377,0.001488,0.011905,0.000992,0.001488,0.036706,0.074901,0.

In [0]:
#CLIENT_ID = 'HAVSYF2EXYE3ZZYGUNAED0X2TA0H3PN4HHLAARFE0SEW41UZ' # your Foursquare ID
#CLIENT_SECRET = 'YLFWFYECMOVPL0JNPVWOR4YR2PETGPBWOBLX4JF1C0EZER2Q' # your Foursquare Secret
CLIENT_ID='A0ZWWRRIUV4Y2BO2W2E4QHTOV3KYV3ALZA0UTSJ5B1F3QNYA'
CLIENT_SECRET='SAESVBMDTVTSOX2YNO5VOFGT1VJVTPOA2B0KWEQY555NJNPS'
VERSION = '20200524'

In [0]:
#function to obtain details of venues in each neighborhood using foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=50
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
boston_venues=getNearbyVenues(names=high_crime['STREET'],
                               latitudes=high_crime['Lat'],
                               longitudes=high_crime['Long'])
boston_venues.head()

ADAMS ST
ALBANY ST
ALLSTATE RD
AMERICAN LEGION HWY
BEACON ST
BLUE HILL AVE
BORDER ST
BOWDOIN ST
BOYLSTON ST
CAMBRIDGE ST
CENTRE ST
COLUMBIA RD
COLUMBUS AVE
COMMONWEALTH AVE
DORCHESTER AVE
DUDLEY ST
GENEVA AVE
HANCOCK ST
HARRISON AVE
HUNTINGTON AVE
HYDE PARK AVE
MASSACHUSETTS AVE
NEW SUDBURY ST
NEWBURY ST
RIVER ST
SUMMER ST
TREMONT ST
W BROADWAY
WARREN ST
WASHINGTON ST


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ADAMS ST,42.305378,-71.061514,Pho Le,42.304420,-71.059102,Vietnamese Restaurant
1,ADAMS ST,42.305378,-71.061514,Reign Drink Lab,42.303961,-71.059008,Café
2,ADAMS ST,42.305378,-71.061514,Pho 2000 Restaurant,42.302148,-71.060053,Vietnamese Restaurant
3,ADAMS ST,42.305378,-71.061514,dbar,42.308343,-71.058305,New American Restaurant
4,ADAMS ST,42.305378,-71.061514,Peggy O'Neils,42.305816,-71.058642,Dive Bar


In [0]:
cols=['STREET','STREET Latitude', 
                  'STREET Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
boston_venues.columns=cols
boston_venues.head()

,STREET,STREET Latitude,STREET Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ADAMS ST,42.305378,-71.061514,Pho Le,42.304420,-71.059102,Vietnamese Restaurant
1,ADAMS ST,42.305378,-71.061514,Reign Drink Lab,42.303961,-71.059008,Café
2,ADAMS ST,42.305378,-71.061514,Pho 2000 Restaurant,42.302148,-71.060053,Vietnamese Restaurant
3,ADAMS ST,42.305378,-71.061514,dbar,42.308343,-71.058305,New American Restaurant
4,ADAMS ST,42.305378,-71.061514,Peggy O'Neils,42.305816,-71.058642,Dive Bar


In [0]:
# one hot encoding of venues
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['STREET'] = boston_venues['STREET'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,STREET,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Business Service,Café,Cambodian Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,...,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Track,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ADAMS ST,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,ADAMS ST,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ADAMS ST,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,ADAMS ST,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ADAMS ST,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#find mean for normalized value
boston_grouped = boston_onehot.groupby('STREET').mean().reset_index()
boston_grouped.head()

,STREET,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Business Service,Café,Cambodian Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,...,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Track,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ADAMS ST,0.0,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00000,0.00000,0.00,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.00,0.0,0.0,0.0,0.0,0.038462,0.00,0.0,0.0,0.0,0.00000,0.076923,0.0,0.038462,0.0,0.0,0.0,0.00000,...,0.076923,0.0,0.0,0.038462,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.192308,0.0,0.0,0.00,0.0
1,ALBANY ST,0.0,0.00,0.05,0.000000,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.00000,0.00000,0.00,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.00000,0.100000,0.0,0.050000,0.0,0.0,0.0,0.00000,...,0.000000,0.0,0.0,0.000000,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
2,ALLSTATE RD,0.0,0.00,0.00,0.047619,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.02381,0.02381,0.00,0.0,0.0,0.00,0.0,0.0,0.02381,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.02381,0.000000,0.0,0.000000,0.0,0.0,0.0,0.02381,...,0.000000,0.0,0.0,0.000000,0.00,0.02381,0.02381,0.02381,0.0,0.0,0.0,0.0,0.02381,0.00,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,AMERICAN LEGION HWY,0.0,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00000,0.00000,0.00,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.25000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.000000,0.0,0.0,0.000000,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,BEACON ST,0.0,0.02,0.00,0.020000,0.00,0.0,0.02,0.0,0.00,0.0,0.0,0.04000,0.00000,0.02,0.0,0.0,0.02,0.0,0.0,0.00000,0.0,0.02,0.0,0.0,0.0,0.0,0.020000,0.00,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.02000,...,0.000000,0.0,0.0,0.020000,0.02,0.02000,0.02000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.12,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.02,0.0


Merge dummy variables of crimes and venue data for clustering

In [0]:
merged=crime_grouped.merge(boston_grouped,on='STREET')
merged.head()

,STREET,Aggravated Assault,Aircraft,Arson,Assembly or Gathering Violations,Auto Theft,Auto Theft Recovery,Ballistics,Biological Threat,Bomb Hoax,Burglary - No Property Taken,Commercial Burglary,Confidence Games,Counterfeiting,Criminal Harassment,Disorderly Conduct,Drug Violation,Embezzlement,Evading Fare,Explosives,Fire Related Reports,Firearm Discovery,Firearm Violations,Fraud,Gambling,HOME INVASION,HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE,Harassment,Harbor Related Incidents,Homicide,INVESTIGATE PERSON,Investigate Person,Investigate Property,Landlord/Tenant Disputes,Larceny,Larceny From Motor Vehicle,License Plate Related Incidents,License Violation,Liquor Violation,Manslaughter,...,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Track,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ADAMS ST,0.017110,0.0,0.0,0.000951,0.016635,0.004753,0.005228,0.0,0.000000,0.000000,0.005228,0.013783,0.009506,0.000475,0.006179,0.048004,0.000475,0.000951,0.000000,0.006654,0.001901,0.003802,0.011882,0.000475,0.0,0.0,0.007605,0.0,0.000475,0.0,0.048479,0.031844,0.000951,0.040875,0.027567,0.003327,0.007129,0.001426,0.000000,...,0.076923,0.0,0.0,0.038462,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.192308,0.0,0.0,0.00,0.0
1,ALBANY ST,0.029332,0.0,0.0,0.000682,0.006139,0.000682,0.001364,0.0,0.000682,0.000000,0.007503,0.004775,0.000682,0.000000,0.016371,0.156889,0.001364,0.001364,0.000000,0.001364,0.000000,0.006821,0.008186,0.000000,0.0,0.0,0.004775,0.0,0.000000,0.0,0.035471,0.011596,0.000682,0.070941,0.013643,0.000000,0.000682,0.006139,0.000000,...,0.000000,0.0,0.0,0.000000,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
2,ALLSTATE RD,0.014264,0.0,0.0,0.000000,0.009760,0.003754,0.000000,0.0,0.000000,0.000000,0.002252,0.026276,0.002252,0.000000,0.003754,0.039039,0.006006,0.001502,0.000000,0.000751,0.000000,0.002252,0.008258,0.000000,0.0,0.0,0.003754,0.0,0.000000,0.0,0.044294,0.006006,0.000000,0.337838,0.030030,0.001502,0.001502,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.00,0.02381,0.02381,0.02381,0.0,0.0,0.0,0.0,0.02381,0.00,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,AMERICAN LEGION HWY,0.016746,0.0,0.0,0.000598,0.009569,0.000000,0.001196,0.0,0.000000,0.000598,0.006579,0.003589,0.016746,0.000598,0.004187,0.041866,0.007775,0.001794,0.000000,0.005383,0.000598,0.007177,0.014354,0.000000,0.0,0.0,0.007177,0.0,0.000000,0.0,0.046053,0.025120,0.002990,0.122010,0.026316,0.001196,0.001794,0.000000,0.000598,...,0.000000,0.0,0.0,0.000000,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,BEACON ST,0.011905,0.0,0.0,0.026786,0.017361,0.004960,0.000496,0.0,0.000000,0.000000,0.003472,0.009921,0.003968,0.000496,0.008433,0.026786,0.001488,0.001488,0.000496,0.008433,0.000992,0.001984,0.026786,0.000000,0.0,0.0,0.009425,0.0,0.000000,0.0,0.038194,0.042163,0.001984,0.083829,0.089782,0.001488,0.008929,0.003968,0.000000,...,0.000000,0.0,0.0,0.020000,0.02,0.02000,0.02000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.12,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.02,0.0


function to return most common values

In [0]:
def return_most_common(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

To form a dataframe that contains top 10 venues of respective areas

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['STREET']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['STREET'] = boston_grouped['STREET']

for ind in np.arange(boston_grouped.shape[0]):
   venues_sorted.iloc[ind, 1:] = return_most_common(boston_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,STREET,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADAMS ST,Vietnamese Restaurant,Grocery Store,Restaurant,Café,Gym / Fitness Center,Food,Pizza Place,Plaza,Rental Car Location,New American Restaurant
1,ALBANY ST,Thrift / Vintage Store,Hotel,Café,Brewery,Mobile Phone Shop,Donut Shop,Concert Hall,Caribbean Restaurant,Music Venue,Sushi Restaurant
2,ALLSTATE RD,Coffee Shop,Deli / Bodega,American Restaurant,Donut Shop,Department Store,Liquor Store,Mattress Store,Electronics Store,Shopping Mall,Shoe Store
3,AMERICAN LEGION HWY,Pizza Place,Liquor Store,Business Service,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Electronics Store,Donut Shop,Dog Run
4,BEACON ST,Spa,Jewelry Store,Pizza Place,Park,Bakery,Dessert Shop,Women's Store,Outdoor Sculpture,Scenic Lookout,Gym


To form a dataframe with top 10 common crimes in respective areas

In [0]:
num_top_crimes = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['STREET']
for ind in np.arange(num_top_crimes):
    try:
        columns.append('{}{} Common crime'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Common crime'.format(ind+1))

# create a new dataframe
crimes_sorted = pd.DataFrame(columns=columns)
crimes_sorted['STREET'] = crime_grouped['STREET']

for ind in np.arange(crime_grouped.shape[0]):
   crimes_sorted.iloc[ind, 1:] = return_most_common(crime_grouped.iloc[ind, :], num_top_crimes)

crimes_sorted.head()

,STREET,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime
0,ADAMS ST,Motor Vehicle Accident Response,Medical Assistance,Other,Verbal Disputes,Investigate Person,Drug Violation,Vandalism,Larceny,Simple Assault,Violations
1,ALBANY ST,Motor Vehicle Accident Response,Drug Violation,Other,Warrant Arrests,Simple Assault,Larceny,Medical Assistance,Investigate Person,Aggravated Assault,Property Lost
2,ALLSTATE RD,Larceny,Motor Vehicle Accident Response,Warrant Arrests,Property Lost,Other,Medical Assistance,Investigate Person,Drug Violation,Larceny From Motor Vehicle,Simple Assault
3,AMERICAN LEGION HWY,Motor Vehicle Accident Response,Larceny,Other,Violations,Property Lost,Investigate Person,Medical Assistance,Simple Assault,Drug Violation,Vandalism
4,BEACON ST,Motor Vehicle Accident Response,Larceny From Motor Vehicle,Larceny,Towed,Medical Assistance,Vandalism,Investigate Property,Investigate Person,Simple Assault,Other


# Clustering of locations with k-means clustering algorithm

In [0]:
# set number of clusters
kclusters = 5

boston_grouped_clustering = merged.drop('STREET', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
merged.insert(0, 'Cluster Labels', kmeans.labels_)

In [0]:
merged[['Lat']]=high_crime[['Lat']]
merged[['Long']]=high_crime[['Long']]
merged=merged[['STREET','Lat','Long','Cluster Labels']]
merged=merged.merge(crimes_sorted,on=['STREET'])
merged=merged.merge(venues_sorted,on=['STREET'])

merged.shape

(30, 24)

In [0]:
merged.head()

,STREET,Lat,Long,Cluster Labels,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADAMS ST,42.305378,-71.061514,0,Motor Vehicle Accident Response,Medical Assistance,Other,Verbal Disputes,Investigate Person,Drug Violation,Vandalism,Larceny,Simple Assault,Violations,Vietnamese Restaurant,Grocery Store,Restaurant,Café,Gym / Fitness Center,Food,Pizza Place,Plaza,Rental Car Location,New American Restaurant
1,ALBANY ST,42.330769,-71.077665,0,Motor Vehicle Accident Response,Drug Violation,Other,Warrant Arrests,Simple Assault,Larceny,Medical Assistance,Investigate Person,Aggravated Assault,Property Lost,Thrift / Vintage Store,Hotel,Café,Brewery,Mobile Phone Shop,Donut Shop,Concert Hall,Caribbean Restaurant,Music Venue,Sushi Restaurant
2,ALLSTATE RD,42.326966,-71.061986,1,Larceny,Motor Vehicle Accident Response,Warrant Arrests,Property Lost,Other,Medical Assistance,Investigate Person,Drug Violation,Larceny From Motor Vehicle,Simple Assault,Coffee Shop,Deli / Bodega,American Restaurant,Donut Shop,Department Store,Liquor Store,Mattress Store,Electronics Store,Shopping Mall,Shoe Store
3,AMERICAN LEGION HWY,42.297950,-71.087744,2,Motor Vehicle Accident Response,Larceny,Other,Violations,Property Lost,Investigate Person,Medical Assistance,Simple Assault,Drug Violation,Vandalism,Pizza Place,Liquor Store,Business Service,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Electronics Store,Donut Shop,Dog Run
4,BEACON ST,42.355052,-71.073907,1,Motor Vehicle Accident Response,Larceny From Motor Vehicle,Larceny,Towed,Medical Assistance,Vandalism,Investigate Property,Investigate Person,Simple Assault,Other,Spa,Jewelry Store,Pizza Place,Park,Bakery,Dessert Shop,Women's Store,Outdoor Sculpture,Scenic Lookout,Gym


In [0]:
address = 'Boston'

geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print(ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Lat'], merged['Long'], merged['STREET'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

[array([0, 1, 2, 3, 4]), array([ 1,  3,  7, 13, 21]), array([ 2,  7, 20, 41, 70]), array([  3,  13,  41,  87, 151]), array([  4,  21,  70, 151, 264])]


# Examining clusters

In [0]:
merged.loc[merged['Cluster Labels']==0,merged.columns[[0]+list(range(4,merged.shape[1]))]]

,STREET,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADAMS ST,Motor Vehicle Accident Response,Medical Assistance,Other,Verbal Disputes,Investigate Person,Drug Violation,Vandalism,Larceny,Simple Assault,Violations,Vietnamese Restaurant,Grocery Store,Restaurant,Café,Gym / Fitness Center,Food,Pizza Place,Plaza,Rental Car Location,New American Restaurant
1,ALBANY ST,Motor Vehicle Accident Response,Drug Violation,Other,Warrant Arrests,Simple Assault,Larceny,Medical Assistance,Investigate Person,Aggravated Assault,Property Lost,Thrift / Vintage Store,Hotel,Café,Brewery,Mobile Phone Shop,Donut Shop,Concert Hall,Caribbean Restaurant,Music Venue,Sushi Restaurant
6,BORDER ST,Larceny,Medical Assistance,Drug Violation,Investigate Person,Motor Vehicle Accident Response,Other,Vandalism,Simple Assault,Larceny From Motor Vehicle,Verbal Disputes,Mexican Restaurant,Pharmacy,Donut Shop,Art Gallery,Chinese Restaurant,Latin American Restaurant,Sandwich Place,Grocery Store,Seafood Restaurant,Liquor Store
10,CENTRE ST,Motor Vehicle Accident Response,Drug Violation,Larceny,Medical Assistance,Investigate Person,Other,Simple Assault,Vandalism,Property Lost,Investigate Property,Liquor Store,Train Station,Spa,Nail Salon,Bar,Bank,Coffee Shop,Indian Restaurant,Mexican Restaurant,Grocery Store
14,DORCHESTER AVE,Motor Vehicle Accident Response,Larceny,Medical Assistance,Simple Assault,Other,Drug Violation,Investigate Person,Vandalism,Warrant Arrests,Property Lost,Vietnamese Restaurant,Café,Liquor Store,Pub,Discount Store,Fast Food Restaurant,Seafood Restaurant,Sandwich Place,Supermarket,Baseball Field
15,DUDLEY ST,Motor Vehicle Accident Response,Other,Larceny,Medical Assistance,Investigate Person,Simple Assault,Violations,Drug Violation,Aggravated Assault,Verbal Disputes,American Restaurant,Café,Thrift / Vintage Store,Music Venue,Pharmacy,African Restaurant,Concert Hall,Brewery,Bike Rental / Bike Share,Mobile Phone Shop
16,GENEVA AVE,Motor Vehicle Accident Response,Larceny,Other,Medical Assistance,Simple Assault,Verbal Disputes,Investigate Person,Drug Violation,Warrant Arrests,Vandalism,Vietnamese Restaurant,Seafood Restaurant,Sandwich Place,Café,Pizza Place,Clothing Store,Discount Store,Pub,Dessert Shop,Metro Station
17,HANCOCK ST,Drug Violation,Investigate Person,Motor Vehicle Accident Response,Towed,Medical Assistance,Other,Verbal Disputes,Vandalism,Investigate Property,Simple Assault,Hotel,American Restaurant,Pizza Place,Plaza,Historic Site,Cocktail Bar,Steakhouse,Mediterranean Restaurant,French Restaurant,Lake
18,HARRISON AVE,Drug Violation,Larceny,Other,Warrant Arrests,Motor Vehicle Accident Response,Simple Assault,Medical Assistance,Investigate Person,Property Lost,Property Found,Pet Store,Hotel,Art Gallery,Dog Run,Park,Japanese Restaurant,Italian Restaurant,Pizza Place,Mexican Restaurant,Gym / Fitness Center
20,HYDE PARK AVE,Motor Vehicle Accident Response,Investigate Property,Drug Violation,Medical Assistance,Investigate Person,Larceny,Other,Property Lost,Simple Assault,Vandalism,American Restaurant,Pizza Place,Bar,Platform,Pharmacy,Park,Convenience Store,Sandwich Place,Business Service,Discount Store


From above dataframe we can observe that the top crimes in this cluster of areas is motor vehicle accident,larceny,drug violation

In [0]:
merged.loc[merged['Cluster Labels']==1,merged.columns[[0]+list(range(4,merged.shape[1]))]]

,STREET,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,ALLSTATE RD,Larceny,Motor Vehicle Accident Response,Warrant Arrests,Property Lost,Other,Medical Assistance,Investigate Person,Drug Violation,Larceny From Motor Vehicle,Simple Assault,Coffee Shop,Deli / Bodega,American Restaurant,Donut Shop,Department Store,Liquor Store,Mattress Store,Electronics Store,Shopping Mall,Shoe Store
4,BEACON ST,Motor Vehicle Accident Response,Larceny From Motor Vehicle,Larceny,Towed,Medical Assistance,Vandalism,Investigate Property,Investigate Person,Simple Assault,Other,Spa,Jewelry Store,Pizza Place,Park,Bakery,Dessert Shop,Women's Store,Outdoor Sculpture,Scenic Lookout,Gym
8,BOYLSTON ST,Larceny,Drug Violation,Warrant Arrests,Simple Assault,Motor Vehicle Accident Response,Other,Property Lost,Medical Assistance,Investigate Person,Aggravated Assault,Gym,Brewery,Coffee Shop,Pizza Place,Bagel Shop,Mexican Restaurant,Donut Shop,Shopping Mall,Liquor Store,Dog Run
9,CAMBRIDGE ST,Motor Vehicle Accident Response,Larceny,Medical Assistance,Investigate Person,Simple Assault,Other,Vandalism,Property Lost,Towed,Violations,Historic Site,Sandwich Place,Italian Restaurant,Coffee Shop,Bakery,Seafood Restaurant,Market,Restaurant,Hotel,American Restaurant
11,COLUMBIA RD,Motor Vehicle Accident Response,Medical Assistance,Other,Drug Violation,Larceny,Investigate Person,Simple Assault,Verbal Disputes,Vandalism,Violations,Italian Restaurant,Track,Seafood Restaurant,Convenience Store,Pub,Bus Stop,Beach,Yoga Studio,Falafel Restaurant,Electronics Store
12,COLUMBUS AVE,Motor Vehicle Accident Response,Larceny,Medical Assistance,Towed,Simple Assault,Drug Violation,Other,Investigate Person,Vandalism,Violations,Caribbean Restaurant,Park,Plaza,Italian Restaurant,Donut Shop,Light Rail Station,Furniture / Home Store,Sandwich Place,Mobile Phone Shop,Gym
13,COMMONWEALTH AVE,Motor Vehicle Accident Response,Towed,Larceny,Medical Assistance,Larceny From Motor Vehicle,Investigate Person,Vandalism,Other,Investigate Property,Property Lost,Clothing Store,Ice Cream Shop,Vietnamese Restaurant,New American Restaurant,Bookstore,Coffee Shop,Seafood Restaurant,Gym / Fitness Center,Fast Food Restaurant,Ramen Restaurant
19,HUNTINGTON AVE,Larceny,Motor Vehicle Accident Response,Medical Assistance,Property Lost,Investigate Person,Other,Simple Assault,Larceny From Motor Vehicle,Vandalism,Aggravated Assault,Hotel,Clothing Store,Italian Restaurant,Shopping Mall,Seafood Restaurant,Ice Cream Shop,Burger Joint,Plaza,Gym,Coffee Shop
23,NEWBURY ST,Larceny,Investigate Person,Towed,Simple Assault,Larceny From Motor Vehicle,Property Lost,Motor Vehicle Accident Response,Other,Vandalism,Commercial Burglary,Spa,Park,Jewelry Store,French Restaurant,Bakery,Plaza,Italian Restaurant,Salad Place,American Restaurant,Hotel
25,SUMMER ST,Larceny,Property Lost,Simple Assault,Other,Motor Vehicle Accident Response,Medical Assistance,Investigate Person,Vandalism,Fraud,Aggravated Assault,Coffee Shop,American Restaurant,Gastropub,Sandwich Place,Bakery,Historic Site,Park,Salad Place,Restaurant,Burger Joint


In [0]:
merged.loc[merged['Cluster Labels']==2,merged.columns[[0]+list(range(4,merged.shape[1]))]]

,STREET,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,AMERICAN LEGION HWY,Motor Vehicle Accident Response,Larceny,Other,Violations,Property Lost,Investigate Person,Medical Assistance,Simple Assault,Drug Violation,Vandalism,Pizza Place,Liquor Store,Business Service,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Electronics Store,Donut Shop,Dog Run


In [0]:
merged.loc[merged['Cluster Labels']==3,merged.columns[[0]+list(range(4,merged.shape[1]))]]

,STREET,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,BOWDOIN ST,Motor Vehicle Accident Response,Medical Assistance,Larceny,Simple Assault,Other,Vandalism,Investigate Person,Verbal Disputes,Violations,Drug Violation,Breakfast Spot,Donut Shop,Discount Store,Yoga Studio,Comic Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store,Dog Run


In [0]:
merged.loc[merged['Cluster Labels']==4,merged.columns[[0]+list(range(4,merged.shape[1]))]]

,STREET,1st Common crime,2nd Common crime,3rd Common crime,4th Common crime,5th Common crime,6th Common crime,7th Common crime,8th Common crime,9th Common crime,10th Common crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,BLUE HILL AVE,Motor Vehicle Accident Response,Drug Violation,Larceny,Investigate Person,Other,Medical Assistance,Simple Assault,Violations,Verbal Disputes,Aggravated Assault,Gym / Fitness Center,Pharmacy,Bar,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Electronics Store,Donut Shop,Dog Run
